In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [3]:
train_df.shape, test_df.shape

((42000, 785), (28000, 784))

In [4]:
from torch.utils.data import Dataset, DataLoader
from torch import tensor
import torch

    
def get_x(df):
    return tensor(df.values[:, :28*28]).reshape(df.shape[0], 1, 28, 28).float()

class ImageDataset(Dataset):
    def __init__(self, df):
        self.X = get_x(df)
        self.Y = torch.zeros(10*df.shape[0]).reshape(df.shape[0], 10)
        for i in range(df.shape[0]):
            self.Y[i][df.iloc[i][-1]] = 1
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, ind):
        return self.X[ind], self.Y[ind]
    
dataset = ImageDataset(train_df)
dataloader = DataLoader(dataset, batch_size=32)

In [5]:
from torch import nn, optim
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(120), 
            nn.ReLU(),
            nn.LazyLinear(84), 
            nn.ReLU(),
            nn.LazyLinear(10)
        )
    
    def forward(self, x):
        return self.fc1(x)

In [6]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [7]:
for epoch in range(30):
    print(epoch)
    correct = 0
    total = 0
    for batch in dataloader:
        x_tensor, y_tensor = batch        
        y_pred = model(x_tensor)        
        loss = criterion(y_pred, y_tensor)
        
        assert not torch.isnan(loss)
            
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [8]:
x = get_x(test_df)
y = model(x)
predicted_classes = torch.argmax(y, dim=1)
result = pd.DataFrame({'ImageId': test_df.index+1, 'Label': predicted_classes})
result.to_csv('sub.csv', index=False)